Tài liệu này mang giấy phép Creative Commons Attribution (CC BY). (c) Nguyễn Ngọc Sáng, Zhukovsky 03/2020.

[@SangVn](https://github.com/SangVn) [@VnCFD](https://vncfdgroup.wordpress.com/)

*Thực hành CFD với Python!*

# <center>VnCFD_2D_v2 !</center>

**VnCFD_2D_v2** là chương trình mô phỏng khí động lực học chuyển động dòng chảy không nhớt trong không gian hai chiều, giải hệ phương trình Euler 2D. So với phiên bản 1 VnCFD_2D_v1, phiên bản hai có nhiều thay đổi. Các thay đổi chính bao gồm:

* Sử dụng Python 3.7
* Tăng tốc độ tính toán lên **gấp 5 lần** bằng cách sử dụng hàm tính dòng được viết trên ngôn ngữ FORTRAN, so với việc sử dụng hàm dòng viết bằng Python.
* Cấu trúc dữ liệu thay đổi để phù hợp với **lưới nhiều blocks**.
* Mở rộng thêm các điều kiện biên để phù hợp với lưới nhiều block, để có thể đặt nhiều điều kiện biên trên một biên, và thêm các điều kiện biên subsonic.
* Rút gọn các lệnh chạy chương trình, việc sử dụng chương trình  trở nên rất thuận tiện. 

**Sau đây ta xem xét cụ thể các thay đổi và cách sử dụng.**

**VnCFD_2D_v2** sử dụng ngôn ngữ **python3.7, numpy1.17, matplotlib3.1** được viết trên **PyCharm Community 2019.3**, sử dụng **Anaconda3 Environment** (xem file `lib/anaconda3_envs_VnCFD_2D_v2.yml`).

# 1. Tăng tốc độ tính toán bằng FORTRAN 

Để tăng tốc độ tính toán chương trình Python ta có nhiều cách khác nhau như sử dụng Cython, Numba [link](https://nbviewer.jupyter.org/github/SangVn/tang_toc_python_code/blob/master/tang_toc_python_code.ipynb) hay sử dụng các hàm được viết bằng FORTRAN. Trong phiên bản v2 VnCFD_2D, hàm tính dòng được viết bằng cả hai ngôn ngữ Python và FORTRAN. Trong module "fluxes.py" có hai hàm tính dòng `flux_roe_python và flux_roe_fortran`. Hàm `flux_roe_fortran` gọi hàm `roe` trong module `fluxes_fortran`. Module FORTRAN không thể gọi trực tiếp trong Python. Để sử dụng nó ta cần chuyển sang file Python bằng cách sử dụng hàm **F2PY**  trong thư viện `numpy`. Thông tin cụ thể tại [https://www.numfys.net/howto/F2PY/]. 

Rõ ràng, chúng ta cần có trình biên dịch `gfortran`. Trên linux sử dụng lệnh:
<br><center>`sudo apt-get install gfortran`.</center></br> 
Để chuyển module FORTRAN sang Python, dùng lệnh:
<br><center> `python -m numpy.f2py -c fluxes_fortran.f90 -m fluxes_fortran`.</center></br>
Xuất hiện module **fluxes_fortran.so** mà Python có thể gọi trực tiếp. Cách gọi giống như gọi module Python thông thường.

So sánh tốc độ tính toán khi sử dụng hai hàm tính dòng này sẽ được xem xét cụ thể trong phần hướng dẫn thực hành.

# 2. Cấu trúc dữ liệu 

Trong version 1 có 4 lớp dữ liệu cơ bản: **Side, Sides, Cell, Cells**, ngoài ra phải tính đến **points** là một `numpy.array`. Version 2 có thêm hai lớp dữ liệu: **Block, Blocks**, points được đổi tên thành **nodes**. Mỗi **Block** chứa ba đối tượng **Sides, Cells** và **nodes**. **Blocks** chứa một hoặc nhiều **Block**.

Để xem phần mô tả các classes này có thể sử dụng các câu lệnh **help(class_name), print(class_name.__doc__)**.

**Cụ thể thông tin của các classes:** 

In [1]:
import lib.data as Classes

## 2.1 Class Cell

In [2]:
print(Classes.Cell.__doc__)


    Lớp dữ liệu Cell chứa  các thông số cơ bản của một ô lưới.
    Parameters
    ----------
    vertices : 4 đỉnh lưới theo thứ tự ngược chiều KĐH dùng để xác định center, volume, size

    Attributes
    ----------
    center: tạo độ tâm
    volume: thể tích
    size:   kích thước
    P:   biến nguyên thủy (rho, u, v, p)
    U:   biến bảo toàn (rho, rho*u, rho*v, e), e = p/(gamma-1) + rho*(u^2 + v^2)/2
    res: biến tổng dòng qua các bề mặt sum(F*S)
    dt:  bước thời gian trong ô lưới.

    Notes
    -----
    Tại thời điểm khởi tạo P, U, res, dt bằng 0
    


## 2.2 Class Cells 

In [3]:
help(Classes.Cells)

Help on class Cells in module lib.data:

class Cells(builtins.object)
 |  Cells(nodes)
 |  
 |  Lớp dữ liệu các ô lưới trong một block (zone).
 |  
 |  Parameters
 |  ----------
 |  nodes : mảng 2D các tọa độ các điểm lưới có kích thước (Nj+1)*(Ni+1)
 |  
 |  Attributes
 |  ----------
 |  size:  kích thước lưới 2D ([Nj, Ni])
 |  len:   tổng số ô lưới (Nj*Ni)
 |  cells: dãy các ô lưới "class Cell".
 |  
 |  Notes
 |  -----
 |  Ô lưới thứ (j,i) (hay thứ j*i) gồm 4 đỉnh [(j,i), (j,i+1), (j+1,i+1), (j+1,i)]
 |  
 |  Methods defined here:
 |  
 |  __getitem__(self, item)
 |      Các phương thức cơ bản để lấy các phần tử của dãy các ô lưới "cells":
 |      :param item: có thể là kiểu tuple (j, i), int j*i hay để getslice [start:stop]
 |      :return: một hoặc một đoạn các ô lưới
 |  
 |  __init__(self, nodes)
 |      Khởi tạo Cells từ mảng nodes.
 |  
 |  new_P(self)
 |      Thực hiện bước lặp: xác định P ở bước thời gian tiếp theo, sử dụng hàm U2P.
 |  
 |  new_U(self, dt)
 |      Thực hiện

## 2.3 Class Side 

In [4]:
print(Classes.Side.__doc__)


    Lớp Side chứa các thông số cơ bản của một bề mặt của ô lưới.

    Parameters
    ----------
    side_vec : vector side = (point2 - point1)

    Attributes
    ----------
    area:   diện tích bề mặt, 2D - chiều dài cạnh
    normal: vector pháp tuyến đơn vị
    cells:  hai ô lưới hai bên trái phải.
            Quy ước: self.cells[0] - ô bên trái, self.cells[1] - ô bên phải
    


## 2.4 Class Sides 

In [5]:
help(Classes.Sides)

Help on class Sides in module lib.data:

class Sides(builtins.object)
 |  Sides(nodes, cells)
 |  
 |  Lớp dữ liệu các bề mặt trong một vùng tính toán block (zone).
 |  Sides gồm hai loại:
 |      + Side bên trong vùng tính toán
 |      + Side trên biên
 |  Quy ước:
 |      Mỗi vùng tính toán block có 4 biên:
 |                      bound_3
 |                     <--------
 |                 ^               ^
 |          bound_0|  inner_sides  |bound_1
 |  
 |                     <--------
 |                      bound_2
 |  Parameters
 |  ----------
 |  nodes : mảng chứa tọa độ các điểm lưới
 |  cells : dãy các ô lưới
 |  
 |  Attributes
 |  ----------
 |  bounds : list
 |          [bound_0, bound_1, bound_2, bound_3]
 |  inner_sides : list
 |          tất cả các sides bên trong
 |  boco_list : list
 |          các điều kiện biên trên 4 biên [boco_bound_0, boco_bound_1, boco_bound_2, boco_bound_3],
 |          mỗi boco_bound_i là một list các điều kiện biên trên biên i.
 |  
 |  Metho

## 2.5 Class Block 

In [6]:
help(Classes.Block)

Help on class Block in module lib.data:

class Block(builtins.object)
 |  Block(name, nodes)
 |  
 |  Lớp dữ liệu Block chứa các dữ liệu của một vùng tính toán (block, zone).
 |  
 |  Parameters
 |  ----------
 |  name  : tên block
 |  nodes : mảng tọa độ các điểm lưới
 |  
 |  Attributes
 |  ----------
 |  BField : file chứa trường khí động 'name.field'
 |  BNodes : các điểm lưới
 |  BCells : các ô lưới "class Cells"
 |  BSides : các mặt "class Sides"
 |  BSize  : kích thước lưới
 |  
 |  Methods defined here:
 |  
 |  __init__(self, name, nodes)
 |      Khởi tạo Block có tên "name", có tọa độ điểm lưới "nodes".
 |  
 |  init_field(self, P_t0)
 |      Thiết lập trường khí động tại thời điểm ban đầu.
 |      :param P_t0 : numpy.array(rho, u, v, p)
 |  
 |  iteration(self, flux_func, dt)
 |      Mỗi bước lặp bao gồm:
 |          set_boco() : xác định các Ghost Ceels
 |          flux_bound_sides() : tính dòng qua các sides trên biên
 |          flux_inner_sides() : tính dòng qua các side

## 2.6 Class Blocks

In [7]:
help(Classes.Blocks)

Help on class Blocks in module lib.data:

class Blocks(builtins.object)
 |  Blocks(meshfile='examples/test1D/test1D.mesh')
 |  
 |  Lớp dữ liệu Blocks chứa tất cả các blocks trong vùng tính toán và có chức năng thực hiên
 |  tất cả các bước tính toán khí động.
 |  
 |  Parameters
 |  ----------
 |  meshfile : file lưới ở định dạng Tecplot
 |  
 |  Attributes
 |  ----------
 |  len:
 |          số lượng các blocks
 |  blocks:
 |          dãy các blocks
 |  time_step_global:
 |          bước thời gian trong toàn bộ vùng tính toán
 |  state_file:
 |          file trạng thái tính toán, chứ hai thông số của bước lặp cuối cùng - iter, time
 |  
 |  Methods defined here:
 |  
 |  __getitem__(self, item)
 |      Lấy phần tử của dãy Blocks.
 |  
 |  __init__(self, meshfile='examples/test1D/test1D.mesh')
 |      Khởi tạo Blocks từ file lưới "meshfile".
 |  
 |  export_block_data(self, filename)
 |      Xuất trường khí động vào file định dạng Tecplot block data.
 |  
 |  init_field(self, P_t0=[29

## 3. Điều kiện biên

Trong version 2 ngoài các điều kiện biên **farfield, supersonic_inflow, supersonic_outflow, no_slip, joint** có thêm các điều kiện biên **inflow** cho cả hai trường hợp trên âm và dưới âm, **outflow** với áp suất tại biên cố định (back pressure), điều kiện biên **symmetry** (giống điều kiện biên  **wall slip** cho dòng không nhớt), và điều kiện **null** dùng cho các test 1D. Các điều kiện biên được xác định thông qua các **bất biến Riemann** của hệ phương trình Euler 2D. Như đã giới thiệu ở phần 2, phần 3 **Thực hành CFD với Python!** [https://github.com/sangvn], hệ phương trình Euler có 3 đường đặc trưng tương ứng 3 trị riêng $$V_n-c, V_n, V_n+c $$ với $V_n, c$ là vận tốc pháp tuyến và vận tốc âm thanh. Tương ứng với 3 đường đặc trưng có 3 **bất biến Riemann là những đại lượng không đổi dọc theo đường đặc trưng**: 
$$R_{-} = V_n - \frac{2c}{\gamma - 1}, R = \frac{p}{\rho^\gamma}, R_{+} = V_n + \frac{2c}{\gamma - 1}$$

Hướng của các đường đặc trưng phụ thuộc vào vận tốc $V_n, c$. Căn cứ hướng các đường đặc trưng hai bên biên, ta xác định được các thông số trên biên. Ví dụ trường hợp như ở hình dưới:

<img src="img/Riemann.jpg" width = "500" >

Ta có: $$R_{b-} = R_{r-}, R_{b} = R_{l}, R_{b+} = R_{l+}.$$

Trường hợp không có đầy đủ thông số hai bên trái phải của biên, ví dụ điều kiện biên **outflow**, ta cần xác định một thông số nào đó trên biên, ví dụ áp suất, vận tốc... Điều kiện biên sẽ được giới thiệu chi tiết trong các phần tiếp theo của **Thực hành CFD với Python!**.

## 4. Chạy chương trình

Nhờ sự thay đổi trong cấu trúc dữ liệu, đặc biệc là có thêm lớp dữ liệu **Blocks**, việc thiết lập thông số bài toán và chạy chương trình mô phỏng trở nên rất ngắn gọn, thuận tiện. Cụ thể về các bước chạy mô phỏng (xem trong module **run.py**): 

* Khởi tạo Blocks: 
        blocks = Blocks()
* Thiết lập điều kiện ban đầu: 
        blocks.init_field()
        
    Nếu muốn chạy tiếp kết quả đã có từ lần chạy trước thì bước này không cần thiết. Ví dụ, lần 1 tính dòng chảy tại thời điểm t=1s, lần 2 - tại thời điểm t=2s, khi đó kết quả lần 1 sẽ là điều kiện ban đầu của lần 2. Chương trình sẽ tự động tìm kiếm kết quả lần 1 được ghi trong file **.field**. Nếu lệnh init_field() được gọi, quá trình tính toán sẽ bắt đầu lại từ thời điểm t=0s nên sẽ tốn nhiều thời gian hơn.
* Thực hiện các bước lặp tính toán:
        blocks.run()
* Xuất dữ liệu:
        blocks.export_block_data(file_name)
        
    Mặc định dữ liệu kết quả cuối cùng sẽ được lưu ở định dạng Tecplot Block Data.
* Biểu diễn kết quả:
        blocks.plot_field(field_name, pfunc)

    Trong đó, field_name có thể là 'rho', 'u', 'v', 'p'; lựa chọn pfunc có thể là 'pcolor' và 'contour'. Bước này chỉ cho cái nhìn sơ bộ về kết quả thu được. Để biểu diễn kết quả, tốt nhất hãy dùng phần mềm **Paraview** đã được giới thiệu trong phần 3 "Thực hành CFD với Python!". 
    
Vấn đề then chốt là thiết lập các thông số bài toán, các thông số quá trình tính toán. Tất cả các thông số này được thiết lập trong module **setting.py**. Ví dụ cụ thể được xét ở phần sau.

# 5. Examples 

Việc thiết lập thông số bài toán và thông số quá trình tính toán, chạy chương trình được thực hiện trong hai module **run.py và setting.py**. Thiết lập module **run.py** rất dễ dàng như giới thiệu ở trên.

Trong module setting.py chú ý hai thông số là <b>boco_list và joint_list</b>.

Ví dụ **boco_list** các điều kiện biên trên một biên, như ở hình dưới (3 điều kiện biên trên 3 đoạn):

<b><center>boco = [(symmetry, id_1, id_2), (no_slip, id_2, id_3), (symmetry, id_3, id_4)]</center></b>

trong đó <b>id</b> là số thứ tự của side đầu tiên và side cuối cùng của **mỗi đoạn**. **ID** tương ứng của side đầu và side cuối của **biên** là **None và None**.

<img src="img/boco.jpg" width="500">

Ví dụ **joint_list** cho hai blocks như hình dưới:

<img src="img/joint.jpg" width="350">

<b><center>joint = [(1, 1, id1_1, id1_2, 2, 0, id2_1, id2_2)]</center></b>

nghĩa là: 
<b><center>[(block 1, bound 1, start_side_id, end_side_id] joint to [block 2, bound 0, start_side_id, end_side_id)].</center></b>

#  5.1 Bài toán dòng chảy trong Nozzle

Đây là một trong các bài toán để kiểm tra CFD code của NASA [https://www.grc.nasa.gov/WWW/wind/valid/cdv/cdv01/cdv01.html]

<img src="img/nozzle.png">

Lưới như trên hình, chỉ có một block kích thước 51x21 điểm lưới.

### Module /examples/nozzle/setting.py:

In [31]:
# coding: utf-8
# Copyright (C) 2019  Nguyen Ngoc Sang, <https://github.com/SangVn>

# Bài toán dòng chảy trong Nozzle

from lib.functions import Pm2P
from lib.fluxes import flux_roe
from lib.boco import *

# Trước hết cần chỉ rõ đường dẫn của file lưới
mesh_file = 'examples/nozzle/cdnozzle.mesh'

# Thiết lập thông số dòng chảy tới:
# Mach = 0.2, T = 293.15 K, p = 101325.0 Pa, góc tấn alfa = 0.0 độ
# Hàm Pm2P xác định các biến nguyên thủy (rho, u, v, p) từ các biến đầu vào trên 
P_freestream = Pm2P(M=0.2, T=293.15, p=101325.0, alf=0.0)

# Đăt điều kiện biên: boco_i = [(name, start_side_index, end_side_index), (...)]
def set_boco():
    # hàm set_boco_const thiết lập thông số dòng chảy tới và áp suất p_exit tại mặt ra (nếu cần).
    # áp suất toàn phần: pt = 104190.585, xét 3 trường hợp p_exit/pt = 0.89, 0.6, 0.16
    set_boco_const(Pfree=P_freestream, pexit=104190.585*0.6)
    
    blk1_bc_0 = [(inflow, None, None)]   # Điều kiện biên bound_0 là inflow
    blk1_bc_1 = [(outflow, None, None)]  # Điều kiện biên bound_1 là outflow 
    blk1_bc_2 = [(symmetry, None, None)] # Điều kiện biên bound_2 là symmetry
    blk1_bc_3 = [(symmetry, None, None)] # Điều kiện biên bound_3 là symmetry (hãy thử no_slip để thấy sự khác biệt)
    # (..., Non, None) nghĩa là bắt đầu từ side đầu tiên tới side cuối cùng trên biên 
    blk1_bc_list  = [blk1_bc_0, blk1_bc_1, blk1_bc_2, blk1_bc_3] # list các điều kiện biên của block 1
    
    boco_list = [blk1_bc_list]
    joint_list = None
    return boco_list, joint_list

boco_list, joint_list = set_boco()

# các thông số khác
CFL = 1.0
time_target = None
iter_target = 200

# thời điểm ghi kết quả giữa chừng
write_field_frequency_time = None
write_field_frequency_iter = 100

# thời điểm hiển thị các thông số cơ bản của một bước
print_frequency_iter = 10

# lựa chọn hàm tính flux, hàm flux_roe_fortran đã được include trong lib.boco
flux_func = flux_roe_fortran

### Chú ý:

Nếu **module setting.py trên nằm cùng folder với module run.py** ta chỉ cần chạy module run.py (**command: python run.py**) và thu được kết quả. Nhưng để thuận tiện cho việc tính toán nhiều bài toán khác nhau ta **cho module setting trên vào trong folder của bài toán**, trong trường hợp này là folder **examples/nozzle/**. Trong folder này sẽ chứa tất cả file dữ liệu của bài toán: **file lưới, file .field, solver.state, file export_block_data và module setting.py**. Đồng thời tạo một **module setting.py khác, nằm cùng folder với run.py**, trong module này include tất cả nội dung của module setting ban đầu, và chỉ đường dẫn tới folder bài toán. 

### Module /setting.py:

In [ ]:
# coding: utf-8
# Copyright (C) 2019  Nguyen Ngoc Sang, <https://github.com/SangVn>

# ví dụ 1:
from examples.nozzle.setting import *
path_dir = 'examples/nozzle/'

# ví dụ 2:
# from examples.wedge.setting import *
# path_dir = 'examples/wedge/'

# ví dụ 3:
# from examples.naca0012.setting import *
# path_dir = 'examples/naca0012/'

### Module /run.py

In [ ]:
# coding: utf-8
# Copyright (C) 2019  Nguyen Ngoc Sang, <https://github.com/SangVn>

from lib.data import Blocks

def run():
    blocks = Blocks()
    blocks.init_field()
    blocks.run()
    blocks.export_block_data('field_bd.dat')
    blocks.plot_field(field='rho', pfunc='pcolor')
    
if __name__ == '__main__':
    run()

## Chạy chương trình

Để chạy và biên dịch chương trình có thể dùng PyCharm, hoặc dùng câu lệnh:

**python run.py**.

Màn hình hiển thị:

In [ ]:
Import mesh from: examples/nozzle/cdnozzle.mesh

The time taken by init_block is 0.058257 seconds!
The time taken by init_field is 0.002172 seconds!
**************************************************
The time taken by read_field is 0.004180 seconds!

iteration: 50, dt: 0.000021, time: 0.001121
iteration: 100, dt: 0.000020, time: 0.002142

write_field at iteration: 100, time: 0.002142

iteration: 150, dt: 0.000020, time: 0.003129
iteration: 200, dt: 0.000019, time: 0.004104

The time taken by solver is 3.033647 seconds!
**************************************************
Write block data to: examples/nozzle/field_bd.dat

### Khác biệt tốc độ khi sử dụng flux_roe_python và flux_roe_fortran 

Cũng với các thông số như trên, nếu sử dụng flux_roe_python, thời gian tính toán sẽ vào khoảng 14 đến 18 giây. Nghĩa là tốc độ tính toán tăng lên khoảng 5 lần.

## Kết quả

Sử dụng Paraview để biểu diễn kết quả thu được cho ba trường hợp: `p_exit/pt = 0.89, 0.6, 0.16`.

<img src="img/nozzle_1.png" width="800">

<img src="img/nozzle_2.png" width="800">

<img src="img/nozzle_3.png" width="800">


<b>Tương tự như trên, các bạn thiết lập và tiến hành mô phỏng cho các bài toán hai chiều khác.</b>

# 5.2 Test 1D

**VnCFD_2D_v2** có thể dùng để giải các bài toán dòng chảy một chiều như trong phần 2 **Thực hành CFD với Python!**. Khi đó thay vì gọi hàm **blocks.init_field()**, cần sử dụng **blocks.init_field_test1D(P_left, P_right)**. Điều kiện biên cho vùng tính toán: **(suppersonic_outflow,  suppersonic_outflow,  null, null)**.

Shock Tube:

<img src="img/test1D_1.png" width="700">

<img src="img/test1D_2.png" width="800">

# 6. Kết luận

**VnCFD_2D_v2** có nhiều điểm vượt trội so với phiên bản v1, nhưng việc đặt điều kiện biên và điều kiện joint còn khá phức tạp.

Phiên bản tiếp theo **VnCFD_2D_v3** sẽ có thêm solver cho hệ phương trình **Navier-Stokes**.

Việc tìm hiểu cấu trúc và cách sử dụng chương trình là một phần của khóa học **Thực hành CFD với Python!**. 

Để hoàn thiện chương trình, rất mong các bạn sử dụng và đóng góp ý kiến.

**Hẹn gặp lại và chúc các bạn thành công!**